# Importing the `libs`

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

## Checking for `GPU`

In [2]:
# check if a GPU is available (if a GPU is available, CUDA would be loaded)
use_cuda = torch.cuda.is_available()

# get a reference to the available GPU instance
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

## Loading the `data`

In [3]:
# number of input data points in each batch
batch_size = 128

# instantiate a DataLoader for training
train_loader = torch.utils.data.DataLoader(
    # download MNIST dataset, and only the Training dataset
    datasets.MNIST('../data', train=True, download=True,
                    # pre-processing that each chunk of input data will go through
                    transform=transforms.Compose([
                        # convert to a Tensor
                        transforms.ToTensor(),
                        # data in the tensor will be normalized to the below
                        # mentioned Mean and Standard Deviation
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    # randomly shuffle input data for each batch
    batch_size=batch_size, shuffle=True)

# instantiate a DataLoader for testing
test_loader = torch.utils.data.DataLoader(
    # download MNIST dataset, and only the Testing dataset this time
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        # convert to a Tensor
                        transforms.ToTensor(),
                        # use the same mu and sigma values as for training data
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 118773154.28it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 3183759.83it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 34290559.02it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9449667.05it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [9]:
class FirstDNN(nn.Module):
    def __init__(self):
        super(FirstDNN, self).__init__()
    
        # Block 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)       # r_in:1, n_in:28, j_in:1, s:1, p:1, r_out:3, n_out:28, j_out:1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)      # r_in:3, n_in:28, j_in:1, s:1, p:1, r_out:5, n_out:28, j_out:1
        self.pool1 = nn.MaxPool2d(2, 2)                   # r_in:5, n_in:28, j_in:1, s:2, p:0, r_out:6, n_out:14, j_out:2
        
        # Block 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     # r_in:6,  n_in:14, j_in:2, s:1, p:1, r_out:10, n_out:14, j_out:2
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    # r_in:10, n_in:14, j_in:2, s:1, p:1, r_out:14, n_out:14, j_out:2
        self.pool2 = nn.MaxPool2d(2, 2)                   # r_in:14, n_in:14, j_in:2, s:2, p:0, r_out:16, n_out:7,  j_out:4
        
        # Block 3
        self.conv5 = nn.Conv2d(256, 512, 3)               # r_in:16, n_in:7, j_in:4, s:1, p:0, r_out:24, n_out:5, j_out:4
        self.conv6 = nn.Conv2d(512, 1024, 3)              # r_in:24, n_in:5, j_in:4, s:1, p:0, r_out:32, n_out:3, j_out:4
        self.conv7 = nn.Conv2d(1024, 10, 3)               # r_in:32, n_in:3, j_in:4, s:1, p:0, r_out:40, n_out:1, j_out:4
    
    def forward(self, x):
        # Block 1 (CONV -> RELU -> CONV -> RELU -> MP)
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        
        # Block 2 (CONV -> RELU -> CONV -> RELU -> MP)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        
        # Block 3 (CONV -> RELU -> CONV -> RELU)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))

        # Final Layer (CONV)
        x = self.conv7(x)
        
        # Flatten and Reshaping it to 10 output (Vector)
        x = x.view(-1, 10)
        
        # Applying Softmax on the vector to get the intensity of 10 different outputs 
        return F.log_softmax(x)


In [10]:
# send the model to the GPU device
model = FirstDNN().to(device)

In [11]:
# print a summary of the above model, given the input_size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


## Defining the Train and Test wrapper function

In [16]:
# Basic workflows for training and testing of the model
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    # tell the model that we are in the training phase
    model.train()

    # Prepare to iterate the progress bar over the input training data loader
    pbar = tqdm(train_loader)

    # 5-step process of training, for each iteration of input data batch
    for batch_idx, (data, target) in enumerate(pbar):
        # initialize the training input data and labels to the device memory
        data, target = data.to(device), target.to(device)
    
        # Step-1: intialize the optimizer parameters so that gradients of other
        # epochs dont mess with the current epoch
        optimizer.zero_grad()
        # Step-2: do a forward pass through the model using the current data batch
        output = model(data)
        # Step-3: compute the loss after the forward pass
        loss = F.nll_loss(output, target)
        # Step-4: compute the gradients for all tensors in the model using backpropagation
        loss.backward()
        # Step-5: iterate over all parameters (tensors) it is supposed to update
        # and use their internally stored grad to update their values
        optimizer.step()
    
        # visual output of the progress
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

def test(model, device, test_loader):
    # tell the model that we are in the testing phase
    model.eval()
    test_loss = 0
    correct = 0

    # tell PyTorch not to update the gradients while doing the following
    with torch.no_grad():
        # iterate over the test Data Loader, with one data batch every iteration
        for data, target in test_loader:
            # initialize the testing input data and labels to the device memory
            data, target = data.to(device), target.to(device)
            # run the test data through the model
            output = model(data)
            # compute the loss and accumulate the loss over all batches of test data
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            # identify the output prediction as the index with the highest
            # log-probability
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # if the prediction is the same as the expected (target),
            # track it in the "correct" counter
            correct += pred.eq(target.view_as(pred)).sum().item()

    # compute the average loss over the test dataset
    test_loss /= len(test_loader.dataset)

    # print a summary of the test run
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:
# setup the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# multi-epoch train-test loop (currently shows only 1 epoch)
for epoch in range(1, 2):
    # train the model, results in model parameters to be adjusted
    train(model, device, train_loader, optimizer, epoch)
    # test the model accuracy with the learnt model parameters
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.010093619115650654 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.15it/s]  



Test set: Average loss: 0.0243, Accuracy: 9938/10000 (99.38%)



## Experiments 

In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.00026630351203493774 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.68it/s]



Test set: Average loss: 0.0263, Accuracy: 9921/10000 (99.21%)



loss=0.0004544091643765569 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.86it/s] 



Test set: Average loss: 0.0256, Accuracy: 9942/10000 (99.42%)



loss=0.00015562029147986323 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0278, Accuracy: 9938/10000 (99.38%)



loss=1.7085976651287638e-06 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.82it/s]



Test set: Average loss: 0.0281, Accuracy: 9941/10000 (99.41%)



loss=2.8099166229367256e-05 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.74it/s]



Test set: Average loss: 0.0302, Accuracy: 9939/10000 (99.39%)



In [23]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.02342689037322998 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.72it/s]   



Test set: Average loss: 0.0302, Accuracy: 9934/10000 (99.34%)



loss=0.000468453363282606 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.62it/s]  



Test set: Average loss: 0.0294, Accuracy: 9922/10000 (99.22%)



loss=0.00047423309297300875 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.78it/s]



Test set: Average loss: 0.0435, Accuracy: 9905/10000 (99.05%)



loss=0.0004801483592018485 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.83it/s] 



Test set: Average loss: 0.0265, Accuracy: 9926/10000 (99.26%)



loss=0.0035322632174938917 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.72it/s] 



Test set: Average loss: 0.0277, Accuracy: 9942/10000 (99.42%)



In [24]:
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=4.304241156205535e-05 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.77it/s] 



Test set: Average loss: 0.0260, Accuracy: 9945/10000 (99.45%)



loss=6.427519110729918e-06 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.82it/s] 



Test set: Average loss: 0.0276, Accuracy: 9948/10000 (99.48%)



loss=8.755607268540189e-06 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.81it/s] 



Test set: Average loss: 0.0287, Accuracy: 9948/10000 (99.48%)



loss=1.3946876606496517e-05 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.88it/s]



Test set: Average loss: 0.0295, Accuracy: 9948/10000 (99.48%)



loss=1.2558134585560765e-05 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.88it/s]



Test set: Average loss: 0.0301, Accuracy: 9948/10000 (99.48%)



In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.04, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=3.104405621456863e-08 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.75it/s] 



Test set: Average loss: 0.0312, Accuracy: 9949/10000 (99.49%)



loss=7.562179575870687e-07 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.74it/s] 



Test set: Average loss: 0.0320, Accuracy: 9950/10000 (99.50%)



loss=1.3535178311485652e-07 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.94it/s]



Test set: Average loss: 0.0327, Accuracy: 9950/10000 (99.50%)



loss=2.0646470147767104e-05 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0332, Accuracy: 9951/10000 (99.51%)



loss=7.450578376477779e-09 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.25it/s] 



Test set: Average loss: 0.0337, Accuracy: 9951/10000 (99.51%)



In [26]:
optimizer = optim.SGD(model.parameters(), lr=0.08, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/tmp/ipykernel_3804/4093577519.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=1.8129628642782336e-07 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.83it/s]



Test set: Average loss: 0.0345, Accuracy: 9952/10000 (99.52%)



loss=9.934105982267738e-09 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.07it/s] 



Test set: Average loss: 0.0352, Accuracy: 9953/10000 (99.53%)



loss=1.0517297823753324e-06 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.80it/s]



Test set: Average loss: 0.0358, Accuracy: 9953/10000 (99.53%)



loss=1.539778651249435e-07 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s] 



Test set: Average loss: 0.0363, Accuracy: 9953/10000 (99.53%)



loss=1.138670313594048e-06 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.87it/s] 



Test set: Average loss: 0.0367, Accuracy: 9953/10000 (99.53%)

